In [20]:
#imports and setup
import requests

time_outs = (10,60)
client_ID = '413APP-001'
with open('.api_info','r') as f:
    client_sec = f.read()
grant_type='client_credentials'
auth_url = 'https://libraryanswers.fullerton.edu/api/1.1/oauth/token'
ticket_url = 'https://libraryanswers.fullerton.edu/api/1.1/ticket/create'
qid ='2306'

In [22]:
# Authorize
post_variables = dict(client_id=client_ID,client_secret=client_sec,grant_type=grant_type)

authorization = requests.post(auth_url, data = post_variables,timeout=time_outs)

#print(authorization.text)
creds = authorization.json()

{"access_token":"f23bae232dbe6d61aa9c74a6e86daf3ca82f9d92","expires_in":86400,"token_type":"Bearer","scope":"app_create"}


In [23]:
#test ticket
ticket_payload = {'quid':'2306',
'pquestion':'This is a test created By DCP;ignore',
'pdetails':'This is a test created By DCP;ignore'
}

ticket_request = requests.post(ticket_url, timeout=time_outs, data = ticket_payload, headers={'Authorization': 'Bearer {}'.format(creds['access_token'])})

if ticket_request.status_code == 200:
    print("ticket created", ticket_request.text)
else:
    print('Fail: ', ticket_request.status_code)

ticket created {"isShared":false,"ticketUrl":"\/admin\/ticket?qid=10717737","claimed":0}


In [24]:
import html
import pandas as pd

In [25]:

# Multi-ticket generation
tickets= pd.read_excel('ATIissuesSample.xlsx')
tickets.loc[tickets.URL.isna(),'URL'] = 'Link Not Found'
tickets["Ticketed"] = False

#only do pollak or not pollak at a time
#ticket_to_make = tickets[~tickets.Owner.str.contains('Pollak')]
ticket_to_make = tickets[tickets.Owner.str.contains('Pollak')]
ticket_to_make.set_index("ID")
for i,row in ticket_to_make.iterrows():
    messageOut = "\nPage\n"
    messageOut += row['URL'] +'\n'
    messageOut += "Issue Category\n"
    messageOut += row['ResultText'] +'\n'
    messageOut += "Issue Details\n"
    messageOut += html.escape(row['issue']) +'\n\n'
        
    ticket_payload = dict(quid=qid, pquestion='TEST IGNORE: ATI Issues for {}'.format(row['Owner']), pdetails = messageOut)

    #print(messageOut)
           
# create the actual tickets
    ticket_request = requests.post(ticket_url, timeout=time_outs, data = ticket_payload, headers={'Authorization': 'Bearer {}'.format(creds['access_token'])})
    if ticket_request.status_code == 200:
        print("ticket created", ticket_request.text)
        tickets.loc[row['ID'],"Ticketed"] = True
    else:
        print('Fail: ',messageOut)
        
tickets.to_excel('ticketing_results.xlsx', index=False)

ticket created {"isShared":false,"ticketUrl":"\/admin\/ticket?qid=10717738","claimed":0}
ticket created {"isShared":false,"ticketUrl":"\/admin\/ticket?qid=10717739","claimed":0}
